In [1]:
from sklearn import datasets
from __future__ import print_function
import numpy as np
from cvxopt import matrix, solvers
from scipy.spatial.distance import cdist
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
cancer_data = datasets.load_breast_cancer()
# show to test record 5th
print(cancer_data.data[5])
print(cancer_data.data.shape)
#target set
print(cancer_data.target)

[1.245e+01 1.570e+01 8.257e+01 4.771e+02 1.278e-01 1.700e-01 1.578e-01
 8.089e-02 2.087e-01 7.613e-02 3.345e-01 8.902e-01 2.217e+00 2.719e+01
 7.510e-03 3.345e-02 3.672e-02 1.137e-02 2.165e-02 5.082e-03 1.547e+01
 2.375e+01 1.034e+02 7.416e+02 1.791e-01 5.249e-01 5.355e-01 1.741e-01
 3.985e-01 1.244e-01]
(569, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 

In [3]:
target = cancer_data.target
target[target == 0] = -1
target

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1,
        1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1, -1,  1,  1,
        1,  1, -1,  1, -1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,
       -1, -1, -1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1,  1,
        1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
       -1,  1, -1, -1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1, -1,
        1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1,
        1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,
        1, -1,  1, -1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1,
        1,  1, -1,  1,  1

In [4]:
X_train, X_test, y_train, y_test = train_test_split(cancer_data.data, target, test_size=0.3, random_state=109)

In [7]:
X0 = X_train[y_train == 1] # class 1
X1 = X_train[y_train == -1] # class -1
X = np.concatenate((X0.T, X1.T), axis = 1) # all data
y = y_train.reshape(1, -1) # labels

### Bài toán ràng buộc

In [9]:
C = 100
# build K
V = np.concatenate((X0.T, -X1.T), axis = 1)
# K ~ P in lib cvxopt.qp
K = matrix(V.T.dot(V))

q = matrix(-np.ones((V.shape[1], 1)))
# build A, b, G, h
G = matrix(np.vstack((-np.eye(V.shape[1]), np.eye(V.shape[1]))))

h = matrix(np.vstack((np.zeros((V.shape[1], 1)), C*np.ones((V.shape[1], 1)))))
A = matrix(y, tc='d')
b = matrix(np.zeros((1, 1)), tc='d')

solvers.options['show_progress'] = False
sol = solvers.qp(K, q, G, h, A, b)

l = np.array(sol['x'])
print('lambda = \n', l.T)

lambda = 
 [[6.63859063e-166 4.53460199e+001 4.84471416e-166 5.69967971e+001
  1.12608201e-165 4.47745818e-165 4.52136508e-165 1.62275282e-165
  8.80016245e-166 2.28697966e-165 1.31645330e-165 1.47202822e-165
  1.24058931e-165 7.83232779e-166 7.24754286e-166 1.75761444e-165
  3.64622125e-164 6.89060474e-166 2.87458668e-165 1.10179422e-165
  1.89691010e-165 4.65853679e-166 7.00520873e+001 7.03465432e-165
  5.78555718e-166 1.25702793e-165 1.75493455e-165 7.60286846e-166
  9.52154896e-166 1.35062505e-165 9.42686818e-166 1.47912606e-165
  3.04301226e-165 1.11016415e-165 1.00000000e+002 8.11744899e-166
  9.10076928e-166 9.99023646e-166 9.81337604e-166 1.02447178e-164
  1.39724290e-165 1.04848909e-165 1.14811521e-165 2.43831684e-165
  9.99064446e-165 2.91398585e-165 8.61639363e-166 7.53569317e-166
  1.19002084e-165 8.42547802e-166 8.51629544e-166 1.91850691e-165
  8.12611100e-166 1.00000000e+002 8.09471863e+001 6.64327918e-166
  2.50028265e-165 1.11497567e-165 3.92854785e-165 1.22815434e-165

In [10]:
S = np.where(l > 1e-5)[0] # support set
S2 = np.where(l < .999*C)[0]

M = list(int(val) for val in S if val in S2) # intersection of two lists

XT = X.T # we need each column to be one data point in this alg
VS = V[:, S]
lS = l[S]
yM = y[0][M]
XM = XT[M, :]

w_dual = VS.dot(lS).reshape(-1, 1)
b_dual = np.mean(yM - XM.dot(w_dual))
print(w_dual.T, b_dual)

[[ 5.70946594e+00  9.65312373e-02 -6.34394230e-01 -1.16806812e-02
  -2.77209522e+00  2.46393887e+00 -4.13759819e+00 -1.15228517e+01
  -2.18593860e+00  2.60007677e+00 -1.93049300e+00  2.04519808e+00
  -2.24488055e-01 -7.57883948e-02 -7.43148505e-01  7.43841152e+00
   1.48107750e+01 -8.48537729e-01  1.84065196e+00  9.52178214e-01
   2.44142098e-01 -4.12761007e-01  8.79228588e-02 -1.68115532e-02
  -6.46207148e+00  6.51016014e+00 -3.94241878e+00 -1.79218750e+01
  -7.36626406e+00  3.55560297e+00]] 0.34855320002577916


In [11]:
def predict(b, w, X):
    return np.sign(w.T @ X + b).flatten()

In [12]:
y_pred_dual = predict(b_dual, w_dual, X)
y_true = y.flatten()

print("Accuracy Score:", accuracy_score(y_true, y_pred_dual))
print("Confusion Matrix:", confusion_matrix(y_true, y_pred_dual))

Accuracy Score: 0.5477386934673367
Confusion Matrix: [[ 58  91]
 [ 89 160]]


### Sử dụng sklearn

In [18]:
# Copy and put code for generate data here
y1 = y.reshape((V.shape[1],))
X1 = X.T # each sample is one row
clf = SVC(kernel = 'linear', C = 100) # use the same C
# if C is small, method will be “SoftMagin SVM”,
# if C is large enough, method is near to hard margin
clf.fit(X1, y1)

w = clf.coef_
w0 = clf.intercept_
print('w = ', w)
print('b = ', w0)

w =  [[ 90.5711651    2.66923308  -7.75987682  -0.57918988   7.60730417
    5.9759342   20.38893734   3.44543048 -18.17156751   4.40196097
  -10.27379107  10.13837259  22.02861186  -1.22924289   5.33335552
   16.19172744   8.78550322   6.42208676  16.75026007   2.95315546
  -28.33944597  -1.10267302   1.26430731   0.32829059  15.20877366
   29.1786765   -5.63169877   5.6649507   80.6031544   16.63789007]]
b =  [-207.79751104]


In [19]:
y_pred = clf.predict(X.T)

print("Accuracy Score:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:", confusion_matrix(y_true, y_pred))

Accuracy Score: 0.6130653266331658
Confusion Matrix: [[ 29 120]
 [ 34 215]]
